
# Atlantic Trading — LNG Optimiser (No dependency on `Optimization model.xlsx`)

This notebook **does not read** your work-in-progress workbook. It ingests only the market files
you uploaded (JKM/TTF/HH forwards, Brent, Baltic Freight, FX) and produces an allocation plan
and a **fresh `interim_deliverable.xlsx`** with Strategy, Expected PnL, and key assumptions.


In [24]:

# %pip install pulp openpyxl --quiet
import warnings
from pathlib import Path
from typing import Dict, List, Tuple
import pandas as pd, numpy as np, matplotlib.pyplot as plt, pulp
import re
import math

warnings.filterwarnings("ignore")

DATA_DIR = Path("./data")
MONTHS = ['Jan-2026','Feb-2026','Mar-2026','Apr-2026','May-2026','Jun-2026']
DAYS_IN_MONTH = {'Jan-2026':31,'Feb-2026':28,'Mar-2026':31,'Apr-2026':30,'May-2026':31,'Jun-2026':30}
_to_mon = {'Jan':'F','Feb':'G','Mar':'H','Apr':'J','May':'K','Jun':'M'}  # futures month code letters
_mon_abbr = ['Jan','Feb','Mar','Apr','May','Jun']
_mon_key = {m: f"{m}-2026" for m in _mon_abbr}

# Contract constants
CARGO_MMBTU = 3_800_000
PURCHASE_ADDER = 2.5
CANCEL_TOLLING = 1.5  # $/MMBtu penalty if cancel at M-2


In [ ]:

# ---------- Load market files only ----------
def safe_read(path: Path, header=0):
    try:
        return pd.read_excel(path, header=header)
    except Exception as e:
        print("⚠️", path.name, e)
        return pd.DataFrame()

jkm_fwd  = safe_read(DATA_DIR/'JKM Spot LNG Forward (Extracted 23Sep25).xlsx')
hh_fwd   = safe_read(DATA_DIR/'Henry Hub Forward (Extracted 23Sep25).xlsx')
ttf_fwd  = safe_read(DATA_DIR/'TTF Forward (Extracted 23Sep25).xlsx')
brent    = safe_read(DATA_DIR/'Brent Oil Historical Prices (Extracted 01Oct25).xlsx')
freight  = safe_read(DATA_DIR/'Baltic LNG Freight Curves Historical .xlsx')
fx_hist  = safe_read(DATA_DIR/'USDSGD FX Spot Rate Historical (Extracted 23Sep25).xlsx')

def extract_jkm_forward(df: pd.DataFrame) -> dict:
    """
    JKM Spot LNG Forward (Extracted 23Sep25).xlsx
    Columns seen: ['Index','Unnamed: 1','Unnamed: 2','Last','Unnamed: 4','Unnamed: 5','Close','Currency']
    We match rows by 'Index' codes like /JKMF26 (Jan-2026), /JKMG26 (Feb-2026), etc.
    """
    if df is None or df.empty:
        return {}
    out = {}
    for mon in _mon_abbr:
        letter = _to_mon[mon]
        pat = rf"/JKM{letter}26"
        hit = df[df['Index'].astype(str).str.contains(pat, na=False, regex=True)]
        if hit.empty:
            continue
        # prefer Close if numeric, else Last
        close = pd.to_numeric(hit['Close'], errors='coerce')
        last  = pd.to_numeric(hit['Last'],  errors='coerce')
        price = close.iloc[0] if not np.isnan(close.iloc[0]) else last.iloc[0]
        if pd.notna(price):
            out[f"{mon}-2026"] = float(price)
    return out
    
def extract_hh_forward(df: pd.DataFrame) -> dict:
    """
    Henry Hub Forward (Extracted 23Sep25).xlsx
    Columns seen: ['Index','Name','Last','% change','Net Change','Close','Currency','Volume']
    We match by 'Name' containing e.g. 'JAN26', 'FEB26', ...
    """
    if df is None or df.empty:
        return {}
    out = {}
    for mon in _mon_abbr:
        tag = f"{mon.upper()}26"
        hit = df[df['Name'].astype(str).str.contains(tag, na=False)]
        if hit.empty:
            continue
        close = pd.to_numeric(hit['Close'], errors='coerce')
        last  = pd.to_numeric(hit['Last'],  errors='coerce')
        price = close.iloc[0] if not np.isnan(close.iloc[0]) else last.iloc[0]
        if pd.notna(price):
            out[f"{mon}-2026"] = float(price)
    return out

def extract_ttf_forward(df: pd.DataFrame) -> dict:
    """
    TTF Forward (Extracted 23Sep25).xlsx
    Columns seen: ['RIC','Index','Unnamed: 2','Last','Unnamed: 4','Unnamed: 5','Close','Currency']
    We match by 'Index' containing e.g. 'TTF JAN26', 'TTF FEB26', ...
    NOTE: Prices look EUR-based; convert later if needed.
    """
    if df is None or df.empty:
        return {}
    out = {}
    for mon in _mon_abbr:
        tag = f"TTF {mon.upper()}26"
        hit = df[df['Index'].astype(str).str.contains(tag, na=False)]
        if hit.empty:
            continue
        close = pd.to_numeric(hit['Close'], errors='coerce')
        last  = pd.to_numeric(hit['Last'],  errors='coerce')
        price = close.iloc[0] if not np.isnan(close.iloc[0]) else last.iloc[0]
        if pd.notna(price):
            out[f"{mon}-2026"] = float(price)
    return out


def extract_forward_map(df: pd.DataFrame, months=('Jan-2026','Feb-2026','Mar-2026','Apr-2026','May-2026','Jun-2026')) -> dict:
    """
    Vendor-specific extractor: works for your JKM, Henry Hub, and TTF forward files.
    Handles both 'months-as-columns' and 'months-in-first-column' layouts.
    """
    if df is None or df.empty:
        return {}

    # Clean & flatten
    df2 = df.dropna(how='all').loc[:, df.notna().any()]
    if isinstance(df2.columns, pd.MultiIndex):
        df2.columns = [" ".join(str(c) for c in tup if str(c) != "nan").strip() for tup in df2.columns]
    else:
        df2.columns = [str(c).strip() for c in df2.columns]

    out = {}

    # --- Case A: months appear as column headers (most likely for these files)
    for c in df2.columns:
        label = str(c)
        for m in months:
            if m.split('-')[0] in label and '26' in label:
                s = pd.to_numeric(df2[c], errors='coerce').dropna()
                if not s.empty:
                    out[m] = float(s.iloc[-1])
    if out:
        return {m: out[m] for m in months if m in out}

    # --- Case B: months are listed down the first column, price in next col
    first_col = df2.columns[0]
    for i, val in enumerate(df2[first_col]):
        s = str(val)
        for m in months:
            if m.split('-')[0] in s and '26' in s:
                # find companion numeric column (first numeric col to right)
                for j in range(1, df2.shape[1]):
                    num = pd.to_numeric(df2.iloc[i, j], errors='coerce')
                    if not np.isnan(num):
                        out[m] = float(num)
                        break
    return {m: out[m] for m in months if m in out}

JKM_FWD = extract_jkm_forward(jkm_fwd)
HH_FWD  = extract_hh_forward(hh_fwd)
TTF_FWD = extract_ttf_forward(ttf_fwd)

# Brent
def extract_brent_price(df: pd.DataFrame) -> float:
    """
    Extract last valid Brent closing price (USD/bbl) from your Excel sheet.
    """
    if df is None or df.empty:
        return 80.0
    df2 = df.copy()
    df2 = df2.dropna(how="all").loc[:, df2.notna().any()]
    df2.columns = [str(c).strip().lower() for c in df2.columns]

    # Look for columns like 'price', 'close', 'last'
    cand = [c for c in df2.columns if any(k in c for k in ['price','close','last'])]
    if cand:
        col = cand[0]
        series = pd.to_numeric(df2[col], errors='coerce').dropna()
        if not series.empty:
            return float(series.iloc[-1])

    # Fallback: last numeric that’s NOT a date-like value
    nums = pd.to_numeric(df2.select_dtypes(include=['number']).stack(), errors='coerce').dropna()
    nums = nums[nums < 500]  # crude filter to remove Excel date serials (>=500 ~ year 1900+)
    if not nums.empty:
        return float(nums.iloc[-1])

    return 80.0

# FX for SG tariff conversions if needed
def extract_usd_sgd_rate(df: pd.DataFrame) -> float:
    """
    Extract last valid USD/SGD spot rate (≈1.3–1.5).
    Filters out percentage or delta columns automatically.
    """
    if df is None or df.empty:
        return 1.35
    df2 = df.copy()
    df2 = df2.dropna(how="all").loc[:, df2.notna().any()]
    df2.columns = [str(c).strip().lower() for c in df2.columns]

    cand = [c for c in df2.columns if any(k in c for k in ['price','close','last','rate','spot'])]
    if cand:
        col = cand[0]
        s = pd.to_numeric(df2[col], errors='coerce')
        s = s[(s > 0.5) & (s < 2.0)]  # realistic FX range
        if not s.empty:
            return float(s.iloc[-1])
    # fallback mean of plausible numbers
    nums = pd.to_numeric(df2.select_dtypes(include=['number']).stack(), errors='coerce')
    nums = nums[(nums > 0.5) & (nums < 2.0)]
    if not nums.empty:
        return float(nums.iloc[-1])
    return 1.35


# ---------- Diagnostic preview: check the first 10 columns of key forward files ----------

def preview_forward_file(df: pd.DataFrame, name: str):
    if df.empty:
        print(f"⚠️ {name}: file appears empty or unreadable.\n")
        return

    print(f"📘 {name} — shape: {df.shape}")
    cols = list(df.columns[:10])  # first 10 columns only
    print("→ Columns:", cols)
    print(df.head(5))
    print("-" * 80)

preview_forward_file(jkm_fwd, "JKM Spot LNG Forward")
preview_forward_file(hh_fwd,  "Henry Hub Forward")
preview_forward_file(ttf_fwd, "TTF Forward")

JKM_FWD, HH_FWD, TTF_FWD, BRENT, USD_SGD


📘 JKM Spot LNG Forward — shape: (62, 8)
→ Columns: ['Index', 'Unnamed: 1', 'Unnamed: 2', 'Last', 'Unnamed: 4', 'Unnamed: 5', 'Close', 'Currency']
     Index      Unnamed: 1  Unnamed: 2    Last  Unnamed: 4  Unnamed: 5  \
0  /JKMX25  LNG JnK NOV5/d         NaN  11.270         NaN         NaN   
1  /JKMZ25  LNG JnK DEC5/d         NaN  11.430         NaN         NaN   
2  /JKMF26  LNG JnK JAN6/d         NaN  11.640         NaN         NaN   
3  /JKMG26  LNG JnK FEB6/d         NaN  11.610         NaN         NaN   
4  /JKMH26  LNG JnK MAR6/d         NaN  11.345         NaN         NaN   

    Close Currency  
0  11.270      USD  
1  11.430      USD  
2  11.640      USD  
3  11.610      USD  
4  11.345      USD  
--------------------------------------------------------------------------------
📘 Henry Hub Forward — shape: (15, 8)
→ Columns: ['Index', 'Name', 'Last', '% change', 'Net Change', 'Close', 'Currency', 'Volume']
   Index             Name   Last  % change  Net Change  Close Currency 

({'Jan-2026': 11.64,
  'Feb-2026': 11.61,
  'Mar-2026': 11.345,
  'Apr-2026': 10.91,
  'May-2026': 10.835,
  'Jun-2026': 10.93},
 {'Jan-2026': 4.165,
  'Feb-2026': 3.964,
  'Mar-2026': 3.607,
  'Apr-2026': 3.473,
  'May-2026': 3.506,
  'Jun-2026': 3.669},
 {'Jan-2026': 33.9},
 2025.0,
 -0.104347218339391)

In [14]:

# ---------- Economics & pricing (no workbook) ----------
DIST_NM = {'SG': 17600, 'JP': 11200, 'CN': 11600}
BOR = 0.00135  # 0.135% per 1000nm mid-point
SG_TARIFF_USD = 0.20  # simple placeholder; can be replaced by SLNG doc if you have one

def delivered_fraction(distance_nm, bor=BOR):
    return max(0.0, 1.0 - (distance_nm/1000.0)*bor)

def freight_usd_per_mmbtu(distance_nm, base_rate=0.12):
    return base_rate*(distance_nm/1000.0)

def purchase_price(month):
    for key in [month, month.replace('-',' '), month.replace('-','/')]:
        if key in HH_FWD: return HH_FWD[key] + 2.5
    return 7.5

def sell_price_sg(month):
    return 0.13*BRENT + (3.0+7.5)/2 + SG_TARIFF_USD

def sell_price_jp(month):
    jkm = JKM_FWD.get(month, 12.0); return jkm + (0.5+1.2)/2 + 0.10

def sell_price_cn(month):
    jkm = JKM_FWD.get(month, 12.0); return jkm + (2.0+3.5)/2 + 0.10

def month_econ(month):
    rows=[]
    fob = purchase_price(month)
    for dest in ['SG','JP','CN']:
        dist = DIST_NM[dest]
        df = delivered_fraction(dist)
        fr = freight_usd_per_mmbtu(dist)
        sell = sell_price_sg(month) if dest=='SG' else sell_price_jp(month) if dest=='JP' else sell_price_cn(month)
        tariff = SG_TARIFF_USD if dest=='SG' else 0.0
        up = sell*df - (fob + fr + tariff)
        rows.append({'month':month,'dest':dest,'delivered_frac':df,'fob':fob,'freight':fr,'tariff':tariff,'sell':sell,'unit_profit':up})
    return pd.DataFrame(rows)

econ_tables = {m: month_econ(m) for m in MONTHS}
pd.concat(econ_tables.values()).head()


,month,dest,delivered_frac,fob,freight,tariff,sell,unit_profit
0,Jan-2026,SG,0.97624,7.5,2.112,0.2,268.70,252.503688
1,Jan-2026,JP,0.98488,7.5,1.344,0.0,12.95,3.910196
2,Jan-2026,CN,0.98434,7.5,1.392,0.0,14.85,5.725449
0,Feb-2026,SG,0.97624,7.5,2.112,0.2,268.70,252.503688
1,Feb-2026,JP,0.98488,7.5,1.344,0.0,12.95,3.910196


In [8]:

# ---------- Optimiser (no workbook) ----------
def optimise(sg_daily=1.4e6, panama_delay_days=0):
    V = CARGO_MMBTU
    demurrage_per_mmbtu = 80_000.0*panama_delay_days/max(V,1)

    m = pulp.LpProblem('atlantic', pulp.LpMaximize)
    y = {mo: pulp.LpVariable(f'lift_{mo}', 0, 1, cat='Binary') for mo in MONTHS}
    x = {(mo,d): pulp.LpVariable(f'alloc_{mo}_{d}', lowBound=0) for mo in MONTHS for d in ['SG','JP','CN']}

    up = {(mo,r['dest']): r['unit_profit']-demurrage_per_mmbtu for mo,df in econ_tables.items() for _,r in df.iterrows()}

    m += pulp.lpSum([x[(mo,d)]*up[(mo,d)] for mo in MONTHS for d in ['SG','JP','CN']]) -          pulp.lpSum([(1-y[mo])*CARGO_MMBTU*CANCEL_TOLLING for mo in MONTHS])

    for mo in MONTHS:
        m += pulp.lpSum([x[(mo,d)] for d in ['SG','JP','CN']]) == y[mo]*V
        m += x[(mo,'SG')] <= sg_daily*DAYS_IN_MONTH[mo]

    m.solve(pulp.PULP_CBC_CMD(msg=False))

    return {'status': pulp.LpStatus[m.status],
            'objective': float(pulp.value(m.objective)),
            'lift': {mo:int(round(y[mo].value())) for mo in MONTHS},
            'alloc': {(mo,d): float(x[(mo,d)].value() or 0.0) for mo in MONTHS for d in ['SG','JP','CN']}}

base_res = optimise()
base_res


{'status': 'Optimal',
 'objective': 5757084086.399999,
 'lift': {'Jan-2026': 1,
  'Feb-2026': 1,
  'Mar-2026': 1,
  'Apr-2026': 1,
  'May-2026': 1,
  'Jun-2026': 1},
 'alloc': {('Jan-2026', 'SG'): 3800000.0,
  ('Jan-2026', 'JP'): 0.0,
  ('Jan-2026', 'CN'): 0.0,
  ('Feb-2026', 'SG'): 3800000.0,
  ('Feb-2026', 'JP'): 0.0,
  ('Feb-2026', 'CN'): 0.0,
  ('Mar-2026', 'SG'): 3800000.0,
  ('Mar-2026', 'JP'): 0.0,
  ('Mar-2026', 'CN'): 0.0,
  ('Apr-2026', 'SG'): 3800000.0,
  ('Apr-2026', 'JP'): 0.0,
  ('Apr-2026', 'CN'): 0.0,
  ('May-2026', 'SG'): 3800000.0,
  ('May-2026', 'JP'): 0.0,
  ('May-2026', 'CN'): 0.0,
  ('Jun-2026', 'SG'): 3800000.0,
  ('Jun-2026', 'JP'): 0.0,
  ('Jun-2026', 'CN'): 0.0}}

In [9]:

# ---------- Summary + Export new 'interim_deliverable.xlsx' ----------
def summarise(res):
    alloc = res['alloc']
    df = pd.DataFrame([{'month':mo,'dest':d,'alloc_mmbtu':v} for (mo,d),v in alloc.items() if v>0])
    up = pd.concat(econ_tables.values(), ignore_index=True)[['month','dest','unit_profit']]
    out = df.merge(up, on=['month','dest'], how='left')
    out['profit_usd'] = out['alloc_mmbtu']*out['unit_profit']
    return out

summary_df = summarise(base_res)
summary_df.head()


,month,dest,alloc_mmbtu,unit_profit,profit_usd
0,Jan-2026,SG,3800000.0,252.503688,959514014.4
1,Feb-2026,SG,3800000.0,252.503688,959514014.4
2,Mar-2026,SG,3800000.0,252.503688,959514014.4
3,Apr-2026,SG,3800000.0,252.503688,959514014.4
4,May-2026,SG,3800000.0,252.503688,959514014.4


In [11]:

# Create a clean deliverable file from scratch (no dependence on your workbook)
import openpyxl
from openpyxl import Workbook

def export_new_deliverable(res, path=Path("./data/interim_deliverable.xlsx")):
    df = summarise(res)
    wb = Workbook()
    ws = wb.active
    ws.title = "Interim Deliverable"

    # Header row
    headers = ["Row", "Jan-2026","Feb-2026","Mar-2026","Apr-2026","May-2026","Jun-2026"]
    ws.append(headers)

    # Strategy row
    strat = ["Strategy"]
    for m in MONTHS:
        s = df[df['month']==m].sort_values('alloc_mmbtu', ascending=False)
        strat.append("; ".join([f"{r.dest}: {r.alloc_mmbtu/1e6:.2f}M" for r in s.itertuples()]) if not s.empty else "Cancel (pay tolling)")
    ws.append(strat)

    # Expected PnL row
    pnl = ["Expected PnL"]
    g = df.groupby('month')['profit_usd'].sum()
    for m in MONTHS:
        pnl.append(float(g.get(m, 0.0)))
    ws.append(pnl)

    # Approach & Logic
    ws.append(["Approach and Logic", "LP model: lift/cancel + allocation; unit_profit = sell*delivered - (FOB+freight+tariff)."])
    ws.append(["Assumptions", "BOR≈0.135%/1000nm; SG tariff≈0.20 USD/mmBtu; HH+2.5; demurrage optional."])
    ws.append(["Other considerations", "Geopolitics; Panama congestion; buyer credit; FX; SLNG capacity."])

    wb.save(path)
    return str(path)

export_path = export_new_deliverable(base_res)
export_path


'data/interim_deliverable.xlsx'